In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/vmeshchaninov/DiffusionTextGeneration-cond-ca/")

In [9]:
import torch
from transformers import AutoTokenizer
from datasets import load_from_disk

In [4]:
from model import Encoder
from model.enc_normalizer import EncNormalizer

In [5]:
from create_config import create_config

In [6]:
config = create_config()

In [25]:
tokenizer_gen = AutoTokenizer.from_pretrained(config.model.encoder_name)
gen_enc_normalizer = EncNormalizer(
    enc_mean_path=config.data.enc_gen_mean,
    enc_std_path=config.data.enc_gen_std,
)
encoder_gen = Encoder(
    config.model.encoder_name,
    enc_normalizer=gen_enc_normalizer,
    is_change_sp_tokens=False,
).eval().cuda()

In [10]:
path = f"{config.data.dataset_path}/test/"
dt = load_from_disk(path)

In [11]:
text = dt["text"][:256]

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [13]:
input = tokenizer(text, return_tensors="pt", padding="max_length", max_length=64, truncation=True, return_special_tokens_mask=True)

In [26]:
encs = encoder_gen(input_ids=input["input_ids"].cuda(), attention_mask=input["attention_mask"].cuda())

In [19]:
embs = encoder_gen.encoder.embeddings.word_embeddings.weight.data.cuda()

In [20]:
torch.norm(embs[tokenizer.cls_token_id])

tensor(1.4820, device='cuda:0')

In [21]:
torch.norm(embs[tokenizer.sep_token_id])

tensor(0.7997, device='cuda:0')

In [22]:
torch.norm(embs[tokenizer.pad_token_id])

tensor(0.8756, device='cuda:0')

In [31]:
mask = 1 - input["special_tokens_mask"].cuda()

In [40]:
norm_encs = torch.norm(encs, dim=-1)

In [58]:
torch.sum(norm_encs * mask) / (torch.sum(mask) + 1e-8)

tensor(27.3713, device='cuda:0', grad_fn=<DivBackward0>)

In [49]:
pad_mask = ((input["input_ids"] == tokenizer.pad_token_id) * 1.).cuda()

In [55]:
torch.sum(norm_encs * pad_mask) / (torch.sum(pad_mask) + 1e-8)

tensor(20.4483, device='cuda:0', grad_fn=<DivBackward0>)

In [51]:
cls_mask = (input["input_ids"] == tokenizer.cls_token_id).cuda() * 1.

In [56]:
torch.sum(norm_encs * cls_mask) / (torch.sum(cls_mask) + 1e-8)

tensor(31.0934, device='cuda:0', grad_fn=<DivBackward0>)

In [53]:
sep_mask = (input["input_ids"] == tokenizer.sep_token_id).cuda() * 1.

In [57]:
torch.sum(norm_encs * sep_mask) / (torch.sum(sep_mask) + 1e-8)

tensor(40.6944, device='cuda:0', grad_fn=<DivBackward0>)

In [39]:
torch.sum(norm_encs * (1 - mask), dim=0) / (torch.sum(1 - mask, dim=0) + 1e-8)

tensor([31.0934,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 42.3054, 20.8381, 20.0870,
        34.7069, 20.7301, 25.7028, 32.2523, 23.3954, 24.8694, 25.2757, 21.6878,
        24.4826, 26.0106, 22.8898, 21.6156, 23.6438, 22.9424, 23.5160, 23.8497,
        23.2279, 22.4620, 22.6546, 22.9527, 21.6776, 21.5629, 21.9630, 21.3755,
        21.6928, 21.6146, 21.0613, 21.0973, 21.0473, 21.3029, 21.0296, 24.5769],
       device='cuda:0', grad_fn=<DivBackward0>)